Esteganografia

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:

import cv2
import numpy as np
import types
from google.colab.patches import cv2_imshow 

In [2]:
def mensagemParaBinario(mensagem):
  if type(mensagem) == str:
    return ''.join([ format(ord(i), "08b") for i in mensagem ])
  elif type(mensagem) == bytes or type(mensagem) == np.ndarray:
    return [ format(i, "08b") for i in mensagem ]
  elif type(mensagem) == int or type(mensagem) == np.uint8:
    return format(mensagem, "08b")
  else:
    raise TypeError("Entrada não suportada")

In [38]:
# Function to hide the secret mensagem into the imagem

def esconderTexto(imagem, mensagem_secreta):

  # calcular o Tamanho em bytes máximo para codificar
  n_bytes = imagem.shape[0] * imagem.shape[1] * 3 // 8
  print("Tamanho em bytes máximo para codificar:", n_bytes)

  #Verifica se o número de bytes para codificar é menor que o tamanho máximo em bytes da imagem
  if len(mensagem_secreta) > n_bytes:
      raise ValueError("Erro! Bytes insuficientes! Tente utilizar uma imagem maior ou uma mensagem menor !!")
  
  mensagem_secreta += "#####" # qualquer string pode ser delimitador

  data_index = 0

  # converte os dados de entrada para binário
  binario_mensagem_secreta = mensagemParaBinario(mensagem_secreta)
  print("binário da mensagem secreta: ", binario_mensagem_secreta)

  data_len = len(binario_mensagem_secreta) # obtém o tamanho dos dados a serem escondidos
  for values in imagem:
      for pixel in values:
          print("pixel ", pixel)
          # converte os valores RGB para o formato binário
          r, g, b = mensagemParaBinario(pixel)
          print('r g b', r,g,b)
          # modifica o bit menos significante apenas se há dados para armazenar
          if data_index < data_len:
              # esconde o dado dentro do bit menos significante do pixel vermelho
              pixel[0] = int(r[:-1] + binario_mensagem_secreta[data_index], 2)
              data_index += 1
          if data_index < data_len:
              # esconde o dado dentro do bit menos significante do pixel verde
              pixel[1] = int(g[:-1] + binario_mensagem_secreta[data_index], 2)
              data_index += 1
          if data_index < data_len:
              # esconde o dado dentro do bit menos significante do pixel azul
              pixel[2] = int(b[:-1] + binario_mensagem_secreta[data_index], 2)
              data_index += 1
          # se os dados forem codificados, quebrar o loop
          if data_index >= data_len:
              break

  return imagem

In [4]:
def obterTexto(imagem):

  dados_binarios = ""
  for valores in imagem:
      for pixel in valores:
          r, g, b = mensagemParaBinario(pixel) #converte os valores de vermelho, verde e azul para o formato binário
          dados_binarios += r[-1] #extraindo dados do bit menos significante do pixel vermelho
          dados_binarios += g[-1] #extraindo dados do bit menos significante do pixel verde
          dados_binarios += b[-1] #extraindo dados do bit menos significante do pixel azul
  # split by 8-bits
  todos_os_bytes = [ dados_binarios[i: i+8] for i in range(0, len(dados_binarios), 8) ]
  # convert from bits to characters
  dados_decodificados = ""
  for byte in todos_os_bytes:
      dados_decodificados += chr(int(byte, 2))
      if dados_decodificados[-5:] == "#####": #check if we have reached the delimeter which is "#####"
          break
  #print(dados_decodificados)
  return dados_decodificados[:-5] #remove the delimeter to show the original hidden mensagem

In [5]:
# esconder texto dentro da imagem
def codificar_texto(): 
  imagem_nome = input("Digite o nome da imagem(com a extensão): ") 
  imagem = cv2.imread(imagem_nome) # Ler a imagem usando OpenCV-Python.
  #Biblioteca para resolver problemas de visão computacional em Python. 
  
  #detalhes da imagem
  print("A forma da imagem é de: ",imagem.shape) #verificar a forma da imagem para calcular o número de bytes
  print("Imagem original: ")
  imagem_novo_tamanho = cv2.resize(imagem, (500, 500)) #mudar o tamanho da imagem
  #cv2_imshow(imagem_novo_tamanho) #mostrar a imagem
      
  data = input("Digite o texto a codificar: ") 
  if (len(data) == 0): 
    raise ValueError('Dados vazios')
  
  filename = input("Digite o nome da nova imagem(com extensão): ")
  imagem_codificada = esconderTexto(imagem, data) # chamar a função que irá esconder os dados dentro da imagem
  cv2.imwrite(filename, imagem_codificada)

 
    

In [36]:
# decodificar o texto da imagem
def decodificar_texto():
  # read the imagem that contains the hidden imagem
  imagem_name = input("Digite o nome da imagem a decodificar (com extensão) :") 
  imagem = cv2.imread(imagem_name) #read the imagem using cv2.imread() 

  print("A imagem esteganografada ficou assim: ")
  resized_imagem = cv2.resize(imagem, (500, 500))  #mudar o tamanho da imagem
  #cv2_imshow(resized_imagem) #mostrar imagem
    
  texto = obterTexto(imagem)
  return texto

In [39]:
# Esteganografia         
def Esteganografia(): 
    a = input("Estenografia de Imagem \n 1. codificar texto \n 2. decodificar texto \n Escolha opção 1 ou 2: ")
    userinput = int(a)
    if (userinput == 1):
      print("\nCodificando....")
      codificar_texto() 
          
    elif (userinput == 2):
      print("\nDecodificando....") 
      print("A mensagem decodificada é: " + decodificar_texto()) 
    else: 
        raise Exception("Insira o valor correto.") 
          
Esteganografia() #codificar imagem

Estenografia de Imagem 
 1. codificar texto 
 2. decodificar texto 
 Escolha opção 1 ou 2: 1

Codificando....
Digite o nome da imagem(com a extensão): /content/sample_data/eu_jaleco.png
A forma da imagem é de:  (256, 256, 3)
Imagem original: 
Digite o texto a codificar: senai
Digite o nome da nova imagem(com extensão): /content/sample_data/eu_jaleco2.png
Tamanho em bytes máximo para codificar: 24576
binário da mensagem secreta:  01110011011001010110111001100001011010010010001100100011001000110010001100100011
pixel  [182 176 175]
r g b 10110110 10110000 10101111
pixel  [183 178 176]
r g b 10110111 10110010 10110000
pixel  [181 177 175]
r g b 10110101 10110001 10101111
pixel  [184 180 176]
r g b 10111000 10110100 10110000
pixel  [184 181 176]
r g b 10111000 10110101 10110000
pixel  [182 179 174]
r g b 10110110 10110011 10101110
pixel  [185 180 178]
r g b 10111001 10110100 10110010
pixel  [181 176 174]
r g b 10110101 10110000 10101110
pixel  [178 173 170]
r g b 10110010 10101101 10101010


In [40]:
Esteganografia() #decodificar imagem

Estenografia de Imagem 
 1. codificar texto 
 2. decodificar texto 
 Escolha opção 1 ou 2: 2

Decodificando....
Digite o nome da imagem a decodificar (com extensão) :/content/sample_data/eu_jaleco2.png
A imagem esteganografada ficou assim: 
A mensagem decodificada é: senai


In [30]:
def diga(texto, vezes):
  for i in range(vezes):
    print(texto)
  return texto +  texto

In [31]:
t = diga("ola", 3)
diga("mundo", 10)

ola
ola
ola
mundo
mundo
mundo
mundo
mundo
mundo
mundo
mundo
mundo
mundo


'mundomundo'

In [32]:
t

'olaola'

In [33]:
a = 10
b = 3.14
c = 'texto'
d = [10,20,30]
senhor_dos_aneis = ['capitulo 1 blablablabla', 'capítulo 2', 'capítulo 3']
#                              0                    1               2   .....  